In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Define the KNN class
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        # TODO: Implement the fit method
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        # TODO: Implement the predict method
        # Implement the predict method: Predict for multiple data points
        predictions = []
        for x in X:
            # Compute distances between x and all examples in the training set
            distances = [self.compute_distance(x, x_train) for x_train in self.X_train]
            # Sort by distance and return the indices of the first k neighbors
            k_indices = np.argsort(distances)[:self.k]
            # Extract the labels of the k nearest neighbor training samples
            k_nearest_labels = [self.y_train[i] for i in k_indices]
            # Find the most common class label
            most_common_label = np.bincount(k_nearest_labels).argmax()
            predictions.append(most_common_label)
        return predictions

    def compute_distance(self, X1, X2):
        # TODO: Implement distance computation based on self.distance_metric
        # Hint: Use numpy operations for efficient computation
        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((X1 - X2)**2))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X1 - X2))


In [ ]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    from sklearn.pipeline import Pipeline
    from sklearn.compose import ColumnTransformer
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import StandardScaler, OneHotEncoder
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # TODO: Implement data preprocessing
    # Handle categorical variables, scale features, etc.
    # Handle missing values only for numeric columns
    X_train = train_data.drop(columns=['CustomerId', 'Surname','Exited'])
    y_train = train_data['Exited']
    X_test = test_data.drop(columns=['CustomerId','Surname'])

    # Separate features and target variable
    X_train = train_data.drop(columns=['CustomerId', 'Surname', 'Exited'])
    y_train = train_data['Exited']
    X_test = test_data.drop(columns=['CustomerId', 'Surname'])

    # Define preprocessing for numerical features: impute missing values, then scale
    numerical_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
        ('scaler', StandardScaler())  # Standardize the numerical features
    ])

    # Define preprocessing for categorical features: impute missing values, then one-hot encode
    categorical_features = ['Geography', 'Gender']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing categorical values with the most frequent value
        ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode the categorical features
    ])

    # Combine preprocessing for numerical and categorical features
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    # Preprocess the training and test data
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    return X_train, y_train, X_test


In [ ]:
# Define cross-validation function
def cross_validate(X, y, knn, n_splits=5):
    # TODO: Implement cross-validation
    # Compute ROC AUC scores
    from sklearn.model_selection import cross_val_score
    from sklearn.neighbors import KNeighborsClassifier

    # Create the KNN model
    knn = KNeighborsClassifier(n_neighbors=5)

    # Perform cross-validation and compute ROC AUC scores
    cv_scores = cross_val_score(knn, X, y, cv=5, scoring='roc_auc')

    # Output the cross-validation scores
    print(f"Cross-validation AUC scores: {cv_scores}")
    print(f"Mean AUC score: {cv_scores.mean()}")

    return cv_scores.mean()

In [ ]:
# Load and preprocess data
X, y, X_test = preprocess_data('train.csv', 'test.csv')

# Create and evaluate model
knn = KNN(k=5, distance_metric='euclidean')

# Perform cross-validation
cv_scores = cross_validate(X, y, knn)

print("Cross-validation scores:", cv_scores)

# TODO: hyperparamters tuning
best_k = 5

# TODO: Train on full dataset with optimal hyperparameters and make predictions on test set
knn = KNN(k=best_k, distance_metric='euclidean')
knn.fit(X, y)
test_predictions = knn.predict(X_test)


# Save test predictions
pd.DataFrame({'id': pd.read_csv('test.csv')['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)

Cross-validation AUC scores: [0.87534258 0.87352836 0.86609007 0.87554447 0.86416553]
Mean AUC score: 0.8709342022922829
Cross-validation scores: 0.8709342022922829
